[![Open in colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/dkedar7/fast_dash/blob/docs/docs/Examples/04_land_cover_map_with_geemap.ipynb)

This notebook is optimized to run in Google Colab.

We'll use the amazing Geemap library for this demo.
Learn more about Geemap at https://geemap.org/.

In [1]:
!pip install fast-dash geemap jupyter_dash

In [2]:
import ee
import geemap.foliumap as geemap

from fast_dash import fastdash, html, Fastify

In [ ]:
# Authenticate Google Earth Engine
ee.Authenticate()

In [3]:
# Define years over which we'll compare land cover
years = ['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019']

# Using Fastify, Fast Dash allows making any Dash component suitable with Fast Dash
iframe_component = Fastify(component=html.Iframe(height="100%"), 
                           component_property="srcdoc")

In [4]:
# Build and deploy!
# If running locally, feel free to drop the mode and port arguments.

@fastdash(theme="Zephyr", mode="inline", port=5000)
def compare_land_cover(year_of_left_layer: str = years, 
                       year_of_right_layer: str = years) -> iframe_component:
    "Compare how land cover in the US changed over the years"

    # Geemap code. Ref: https://huggingface.co/spaces/giswqs/geemap/blob/main/app.py
    Map = geemap.Map(center=(40, -100), zoom=4, height=600)

    nlcd_left = ee.Image(
        f"USGS/NLCD_RELEASES/2019_REL/NLCD/{year_of_left_layer}"
    ).select("landcover")
    nlcd_right = ee.Image(
        f"USGS/NLCD_RELEASES/2019_REL/NLCD/{year_of_right_layer}"
    ).select("landcover")

    left_layer = geemap.ee_tile_layer(nlcd_left, {}, f"NLCD {year_of_left_layer}")
    right_layer = geemap.ee_tile_layer(nlcd_right, {}, f"NLCD {year_of_right_layer}")

    Map.split_map(left_layer, right_layer)

    # Convert to HTML
    land_cover_map = Map.to_html()

    return land_cover_map